# Crypto Token Correlation Heatmep

### Fetch Tokens Data from Coingecko

In [1]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd

load_dotenv()  # Load environment variables from .env

api_key = os.getenv('GECKO_API')  # Get API key from .env
coin_id = 'bitcoin'
vs_currency = 'usd'
days = '30'

url = f'https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart'
params = {
    'vs_currency': vs_currency,
    'days': days
}
headers = {
    'accept': 'application/json',
    'x-cg-pro-api-key': api_key
}

response = requests.get(url, headers=headers, params=params)

### Convert Result into a DataFrame

In [2]:
data = response.json()

# Convert 'prices' to DataFrame
df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

print(df.head())

                timestamp          price
0 2025-07-07 10:03:38.255  108837.969514
1 2025-07-07 11:04:34.959  108691.181357
2 2025-07-07 12:03:23.015  108648.140157
3 2025-07-07 13:03:46.730  108343.933685
4 2025-07-07 14:03:46.463  108435.957397


### Create a Function

In [3]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import joblib

load_dotenv()  # Load environment variables from .env

api_key = os.getenv('GECKO_API')  # Get API key from .env
coin_id = 'bitcoin'
vs_currency = 'usd'
days = '30'

def fetch_token_price_history(token_ids, vs_currency='usd', days='30'):
    """
    Fetch historical price data for multiple tokens from CoinGecko Pro API.
    
    Args:
        token_ids (list): List of token ids (e.g., ['bitcoin', 'ethereum']).
        vs_currency (str): The target currency (default 'usd').
        days (str): Number of days to fetch (default '30').
    
    Returns:
        pd.DataFrame: DataFrame with timestamps as index and tokens as columns.
    """
    load_dotenv()
    api_key = os.getenv('GECKO_API')
    price_dfs = []
    for coin_id in token_ids:
        
        url = f'https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart'
        params = {
            'vs_currency': vs_currency,
            'days': days
        }
        headers = {
            'accept': 'application/json',
            'x-cg-pro-api-key': api_key
        }
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        df = pd.DataFrame(data['prices'], columns=['timestamp', coin_id])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        df = df.resample('1h').last()  # or '1D' for daily
        price_dfs.append(df)

    # Merge all dataframes on timestamp
    result_df = pd.concat(price_dfs, axis=1)
    return result_df

In [4]:
# Example: Define tokens directly for notebook use
tokens = ['bitcoin', 'ethereum', 'solana', 'ripple']  # Add more as needed

# Example: Get tokens from user input
# tokens = input("Enter token IDs separated by commas (e.g., bitcoin,ethereum,solana): ").split(",")
# tokens = [t.strip() for t in tokens]

df_prices = fetch_token_price_history(tokens, days='30')
print(df_prices.head(5000))

# Save df_prices using joblib
save_dir = 'C:/Users/FFFO CASHIER PT/OneDrive/Desktop/Hands-on Projects/database'
os.makedirs(save_dir, exist_ok=True)
joblib.dump(df_prices, os.path.join(save_dir, "df_prices.joblib"))

                           bitcoin     ethereum      solana    ripple
timestamp                                                            
2025-07-07 10:00:00  108837.969514  2571.150071  152.183418  2.278798
2025-07-07 11:00:00  108691.181357  2560.254636  152.311042  2.286592
2025-07-07 12:00:00  108648.140157  2563.630487  152.945509  2.280670
2025-07-07 13:00:00  108343.933685  2549.563672  151.789549  2.273814
2025-07-07 14:00:00  108435.957397  2563.551118  152.521251  2.321212
...                            ...          ...         ...       ...
2025-08-06 05:00:00  113469.766853  3575.691188  162.368898  2.913404
2025-08-06 06:00:00  114133.761344  3631.745831  164.014482  2.943329
2025-08-06 07:00:00  114083.753597  3632.919456  164.052665  2.949705
2025-08-06 08:00:00  114200.392067  3638.585571  164.178826  2.953423
2025-08-06 09:00:00  114088.965348  3629.616934  164.188570  2.946290

[720 rows x 4 columns]


['C:/Users/FFFO CASHIER PT/OneDrive/Desktop/Hands-on Projects/database\\df_prices.joblib']

In [6]:

save_dir = r'C:\Users\FFFO CASHIER PT\OneDrive\Desktop\Hands-on Projects\database'
os.makedirs(save_dir, exist_ok=True)
joblib.dump(df_prices, os.path.join(save_dir, "df_prices.joblib"))

['C:\\Users\\FFFO CASHIER PT\\OneDrive\\Desktop\\Hands-on Projects\\database\\df_prices.joblib']